## 3. Melhores Práticas de Particionamento

**✓ Bom:**
- Particionar por colunas filtradas frequentemente (date, region)
- 5-10 GB por partição (ideal)
- Cardinalidade baixa-média (10-1000 valores únicos)

**✗ Evite:**
- Particionar por colunas únicas (customer_id)
- Partições muito pequenas (< 10MB) ou grandes (> 10GB)
- Muitos níveis de partição (> 3)

In [ ]:
import duckdb
import time

con = duckdb.connect()

# Query com partition filter
start = time.time()
result = con.execute("""
    SELECT COUNT(*), AVG(amount)
    FROM delta_scan('./sales_partitioned')
    WHERE year = 2024 AND month = 1
""").fetchone()
elapsed = time.time() - start

print(f"Resultado: {result}")
print(f"Tempo: {elapsed:.3f}s")
print("\n✓ Apenas partição Jan/2024 foi lida")

## 2. Verificar Partition Pruning

Queries com filtros de partição leem menos dados.

In [ ]:
import duckdb
from deltalake import write_deltalake
import pandas as pd

con = duckdb.connect()

# Criar dados
df = con.execute("""
    SELECT
        i as order_id,
        ['US', 'UK', 'BR', 'JP'][i % 4 + 1] as country,
        DATE '2024-01-01' + (i % 365) * INTERVAL '1 day' as order_date,
        random() * 1000 as amount
    FROM range(100000) t(i)
""").df()

# Adicionar colunas de partição
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month

# Escrever particionado
write_deltalake(
    "./sales_partitioned",
    df,
    partition_by=["year", "month"],
    mode="overwrite"
)

print("✓ Tabela particionada criada!")
print(f"  Linhas: {len(df):,}")
print("  Partições: year/month")

## 1. Criar Tabela Particionada

In [ ]:
# Instalação
%pip install duckdb deltalake pyarrow pandas -q

# Capítulo 08: Particionamento e Data Skipping

Estratégias de particionamento para otimizar queries e reduzir I/O.

# Capitulo 08 Particionamento Data Skipping

Notebook gerado automaticamente a partir do código fonte python.
